# Open field video cropping

Records manual square crop coordinates for arena videos, applies those coordinates in batch with `ffmpeg`, and stores accept/decline decisions after frame-level visual review.

### Analysis steps
1. Display each source video frame, place a square crop ROI interactively, and write selected coordinates to `crop_coordinates.csv`.
2. Match each video day tag (`PD##`) to saved coordinates and crop each video to `*_crop.mp4` with `ffmpeg`.
3. Display the first frame of each cropped video and write manual accept/decline labels to `crop_review.csv`.


### Step 1: Select crop coordinates

Displays each source frame for interactive ROI placement, constrains box placement to valid bounds, and writes saved crop coordinates to `crop_coordinates.csv`.


In [ ]:
# Find bounding boxes ("s" to save, "q" to quit)

import cv2
import os
import csv
import numpy as np

video_dir = r"D:\CK3_open_field\videos\PD11"
output_csv = os.path.join(video_dir, "crop_coordinates.csv")
box_size = 700
border = 100

results = []

def mouse_handler(event, x, y, flags, param):
    """Track mouse drag events and clamp the preview box within valid bounds."""
    state = param['state']
    frame_w, frame_h = param['dims']
    x = np.clip(x, border, border + frame_w - box_size)
    y = np.clip(y, border, border + frame_h - box_size)

    if event == cv2.EVENT_LBUTTONDOWN:
        state['dragging'] = True
        state['preview_x'] = x
        state['preview_y'] = y
    elif event == cv2.EVENT_MOUSEMOVE and state['dragging']:
        state['preview_x'] = x
        state['preview_y'] = y
    elif event == cv2.EVENT_LBUTTONUP:
        state['dragging'] = False
        state['final_x'] = x
        state['final_y'] = y

video_files = [f for f in os.listdir(video_dir) if f.endswith('.mp4')]
for fname in video_files:
    path = os.path.join(video_dir, fname)
    cap = cv2.VideoCapture(path)
    ret, frame = cap.read()
    cap.release()
    if not ret:
        print(f"Could not read: {fname}")
        continue

    frame_h, frame_w = frame.shape[:2]
    padded = cv2.copyMakeBorder(frame, border, border, border, border, cv2.BORDER_CONSTANT, value=(0, 0, 0))

    state = {
        'dragging': False,
        'preview_x': 0,
        'preview_y': 0,
        'final_x': None,
        'final_y': None
    }

    cv2.namedWindow("Select Crop")
    cv2.setMouseCallback("Select Crop", mouse_handler, {'state': state, 'dims': (frame_w, frame_h)})

    while True:
        display = padded.copy()
        if state['dragging'] or state['final_x'] is not None:
            x = state['preview_x'] if state['dragging'] else state['final_x']
            y = state['preview_y'] if state['dragging'] else state['final_y']
            cv2.rectangle(display, (x, y), (x + box_size, y + box_size), (0, 255, 0), 2)

        cv2.imshow("Select Crop", display)
        key = cv2.waitKey(1) & 0xFF
        if key == ord('s') and state['final_x'] is not None:
            crop_x = state['final_x'] - border
            crop_y = state['final_y'] - border
            results.append([fname, crop_x, crop_y, box_size])
            break
        elif key == ord('q'):
            break

    cv2.destroyAllWindows()

with open(output_csv, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['filename', 'x', 'y', 'box_size'])
    writer.writerows(results)


### Step 2: Batch crop videos

Reads saved crop coordinates, maps each `PD##` video to its crop box, runs `ffmpeg` crop commands, and writes cropped outputs as `*_crop.mp4`.


In [ ]:
# Crop videos using the coordinates from the CSV output in the previous step

import os
import re
import csv
import subprocess

video_dir = r"D:\CK3_open_field\videos\PD11\diff"
output_dir = os.path.join(video_dir, "cropped_videos")
os.makedirs(output_dir, exist_ok=True)

crop_csv = os.path.join(video_dir, "crop_coordinates.csv")

# Map day tags (for example `PD11`) to crop coordinates `(x, y, box_size)`.
pd_crop_map = {}

with open(crop_csv, newline='') as f:
    reader = csv.DictReader(f)
    for row in reader:
        match = re.search(r'PD\d{1,2}', row['filename'])
        if match:
            pd_key = match.group()
            pd_crop_map[pd_key] = (int(row['x']), int(row['y']), int(row['box_size']))

for fname in os.listdir(video_dir):
    if not fname.lower().endswith(".mp4"):
        continue

    match = re.search(r'PD\d{1,2}', fname)
    if not match:
        print(f"Skipping (no PD tag): {fname}")
        continue

    pd_key = match.group()
    if pd_key not in pd_crop_map:
        print(f"No crop info for {pd_key}, skipping {fname}")
        continue

    x, y, box_size = pd_crop_map[pd_key]
    input_path = os.path.join(video_dir, fname)
    output_name = os.path.splitext(fname)[0] + "_crop.mp4"
    output_path = os.path.join(output_dir, output_name)

    cmd = [
        "ffmpeg",
        "-i", input_path,
        "-filter:v", f"crop={box_size}:{box_size}:{x}:{y}",
        "-c:v", "libx264",
        "-crf", "18",
        "-preset", "fast",
        "-c:a", "copy",
        output_path
    ]

    print(f"Cropping {fname} using {pd_key} box ({x}, {y}, size {box_size}) → {output_name}")
    subprocess.run(cmd, stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT)


### Step 3: Review cropped outputs

Displays the first frame of each cropped recording, captures manual `accept` or `decline` decisions, and writes review labels to `crop_review.csv`.


In [ ]:
# Review first frame of cropped videos ("A = Accept, D = Decline")

import os
import cv2
import csv

cropped_dir = r"D:\CK3_open_field\videos\cropped_videos"
output_csv = os.path.join(cropped_dir, "crop_review.csv")

results = []

video_files = [f for f in os.listdir(cropped_dir) if f.lower().endswith(".mp4")]

for fname in video_files:
    path = os.path.join(cropped_dir, fname)
    cap = cv2.VideoCapture(path)
    ret, frame = cap.read()
    cap.release()

    if not ret:
        print(f"Could not read frame from {fname}, skipping.")
        continue

    # Show window in original size, placed safely near top-left corner
    cv2.namedWindow("Review Frame", cv2.WINDOW_AUTOSIZE)
    h, w = frame.shape[:2]
    cv2.moveWindow("Review Frame", 100, 50)
    cv2.imshow("Review Frame", frame)

    print(f"Reviewing: {fname} (A = Accept, D = Decline)")

    while True:
        key = cv2.waitKey(0) & 0xFF
        if key == ord('a'):
            results.append([fname, True])
            break
        elif key == ord('d'):
            results.append([fname, False])
            break

    cv2.destroyAllWindows()

# Save results to CSV
with open(output_csv, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['filename', 'ACCEPT'])
    writer.writerows(results)

print(f"\nReview complete. Results saved to: {output_csv}")
